In [2]:
import requests

def get_conceptnet_relationships(term, lang="en"):
    # Query the ConceptNet API
    url = f"http://api.conceptnet.io/c/{lang}/{term}"
    response = requests.get(url).json()

    # Extract edges (relationships)
    edges = response.get('edges', [])
    relationships = []
    for edge in edges:
        # Get relationship type, start, and end nodes
        rel = edge['rel']['label']  # Relationship type
        start = edge['start']['label']  # Start node
        end = edge['end']['label']  # End node
        surface_text = edge.get('surfaceText', "No surface text provided")
        
        # Append to the relationships list
        relationships.append({
            "start": start,
            "relationship": rel,
            "end": end,
            "surfaceText": surface_text
        })
    
    return relationships

# Example usage
word = "car"
relationships = get_conceptnet_relationships(word)

# Print all relationships
print(f"Relationships for the word '{word}':\n")
for rel in relationships:
    print(f"{rel['start']} ({rel['relationship']}) {rel['end']}")
    print(f"  Surface Text: {rel['surfaceText']}")
    print()


Relationships for the word 'car':

drive (RelatedTo) car
  Surface Text: [[drive]] is related to [[car]]

a car (AtLocation) the city
  Surface Text: You are likely to find [[a car]] in [[the city]]

a car (AtLocation) a parking lot
  Surface Text: You are likely to find [[a car]] in [[a parking lot]]

car (RelatedTo) vehicle
  Surface Text: [[car]] is related to [[vehicle]]

A car (CapableOf) go fast
  Surface Text: [[A car]] can [[go fast]]

a car (CapableOf) crash
  Surface Text: [[a car]] can [[crash]]

a car (HasA) seats
  Surface Text: [[a car]] has [[seats]]

a car (AtLocation) the repair shop
  Surface Text: *Something you find at [[the repair shop]] is [[a car]]

A car (CapableOf) roll over
  Surface Text: [[A car]] can [[roll over]]

motor (RelatedTo) car
  Surface Text: [[motor]] is related to [[car]]

car (RelatedTo) automobile
  Surface Text: [[car]] is related to [[automobile]]

a car (CapableOf) slow down
  Surface Text: [[a car]] can [[slow down]]

A tire (PartOf) a car